In [4]:
import numpy as np

data = np.loadtxt("flight_data.csv", delimiter=",")
print(data)


ValueError: could not convert string 'date' to float64 at row 0, column 1.

In [8]:
for i in range(1, len(df)-1):
    if i % 10 > 0:
        df = df.drop([i])

flight_loginterval = int(df["time"].iloc[1][-2:]) - \
    int(df["time"].iloc[0][-2:])


def calculate_distance(row):
    return geodesic((row["previous_latitude"], row["previous_longitude"]), (row["latitude"], row["longitude"])).meters


prev_total_distance = 0


def caluculate_total_distance(row):
    global prev_total_distance
    prev_total_distance += row["distance_m"]
    return prev_total_distance

In [9]:

df["previous_latitude"] = df["latitude"].shift(1)
df["previous_longitude"] = df["longitude"].shift(1)
df["time"] = pd.to_datetime(df["time"])
df["elapsed_time"] = (df["time"] - df["time"].iloc[0]).dt.total_seconds()
df.fillna(0, inplace=True)
df["distance_m"] = df.apply(calculate_distance, axis=1)
df["total_distance_m"] = df.apply(
    lambda row: caluculate_total_distance(row), axis=1)
df["climb_m"] = df["gps_altitude_m"].diff()
df["climb_rate_m/s"] = df["climb_m"] / flight_loginterval
df["speed_km/s"] = (df["distance_m"]/1000) / (flight_loginterval/3600)
df["distance_from_start_m"] = df.apply(lambda row: geodesic(
    (df["latitude"].iloc[0], df["longitude"].iloc[0]), (row["previous_latitude"], row["previous_longitude"])).meters, axis=1)
df.to_csv("flight_data.csv", index=False)
df.head()

C:\Users\serka\AppData\Local\Temp\ipykernel_13636\254540318.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["time"] = pd.to_datetime(df["time"])


,time,latitude,longitude,gps_altitude_m,pressure_altitude_m,previous_latitude,previous_longitude,elapsed_time,distance_m,total_distance_m,climb_m,climb_rate_m/s,speed_km/s,distance_from_start_m
0,2025-03-22 09:11:21,40.031450,32.328550,1142,1147,0.000000,0.000000,0.0,5.512940e+06,5.512940e+06,NaN,NaN,1.984659e+06,5.512940e+06
10,2025-03-22 09:11:31,40.031450,32.328550,1142,1149,40.031450,32.328550,10.0,0.000000e+00,5.512940e+06,0.0,0.0,0.000000e+00,0.000000e+00
20,2025-03-22 09:11:41,40.031567,32.328267,1137,1140,40.031450,32.328550,20.0,2.743475e+01,5.512968e+06,-5.0,-0.5,9.876508e+00,0.000000e+00
30,2025-03-22 09:11:51,40.031900,32.327767,1147,1149,40.031567,32.328267,30.0,5.649079e+01,5.513024e+06,10.0,1.0,2.033668e+01,2.743475e+01
40,2025-03-22 09:12:01,40.032700,32.328383,1139,1141,40.031900,32.327767,40.0,1.032515e+02,5.513128e+06,-8.0,-0.8,3.717053e+01,8.346839e+01


In [10]:
# Example: Predict climb rate
x = df[["distance_m", "gps_altitude_m", "speed_km/s"]].values
y = df["climb_rate_m/s"].values

In [11]:

# Normalize data
scaler = StandardScaler()
x = scaler.fit_transform(x)

# Split data
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [12]:
model = models.Sequential([
    layers.Dense(64, activation='relu', input_shape=(x_train.shape[1],)),
    layers.Dropout(0.2),
    layers.Dense(64, activation='relu'),
    layers.Dense(1)  # Output layer for regression
])

model.compile(optimizer='adam', loss='mse', metrics=['mae'])

# Train the model
history = model.fit(x_train, y_train, epochs=50, batch_size=32, validation_data=(x_test, y_test))
model.save("climb_rate_model.h5")

e:\Wolf\Belgeler\Projelerim\Paragliding_Flight_Analyses\.venv\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
11/11 ━━━━━━━━━━━━━━━━━━━━ 4s 19ms/step - loss: 2.0059 - mae: 1.1703 - val_loss: nan - val_mae: nan
Epoch 2/50
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 1.8785 - mae: 1.1071 - val_loss: nan - val_mae: nan
Epoch 3/50
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 1.8738 - mae: 1.1131 - val_loss: nan - val_mae: nan
Epoch 4/50
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 1.9015 - mae: 1.1129 - val_loss: nan - val_mae: nan
Epoch 5/50
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 2.0732 - mae: 1.1887 - val_loss: nan - val_mae: nan
Epoch 6/50
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 2.0864 - mae: 1.1824 - val_loss: nan - val_mae: nan
Epoch 7/50
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 1.9990 - mae: 1.1667 - val_loss: nan - val_mae: nan
Epoch 8/50
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 1.9345 - mae: 1.1233 - val_loss: nan - val_mae: nan
Epoch 9/50
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 1.8034 - mae: 1.0877 - val_loss: nan - val_mae: nan
Epoch 10/

In [13]:
loaded_model = models.load_model("climb_rate_model.h5")
# df["predicted_climb_rate_m/s"] = loaded_model.predict(X)


TypeError: Could not locate function 'mse'. Make sure custom classes are decorated with `@keras.saving.register_keras_serializable()`. Full object config: {'module': 'keras.metrics', 'class_name': 'function', 'config': 'mse', 'registered_name': 'mse'}